In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from transformers import SwinModel, SwinConfig

# Custom Dataset Class
class CovidPneumoniaDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['covid', 'pneumonia']
        self.images = []
        self.labels = []
        
        # Load images and labels
        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Custom Model with Swin Tiny Transformer
class SwinBinaryClassifier(nn.Module):
    def __init__(self):
        super(SwinBinaryClassifier, self).__init__()
        # Load pretrained Swin Tiny
        self.swin = SwinModel.from_pretrained('microsoft/swin-tiny-patch4-window7-224')
        # Custom head for binary classification
        self.head = nn.Sequential(
            nn.Linear(768, 256),  # Swin Tiny output is 768
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)  # 2 classes: covid and pneumonia
        )
    
    def forward_features(self, x):
        return self.swin(x).last_hidden_state[:, 0, :]  # Get CLS token
    
    def forward(self, x):
        features = self.forward_features(x)
        return self.head(features)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

# Load dataset
data_dir = "/kaggle/input/covid-pneumonia-lus-images/covid_pneumonia"  # Replace with your dataset path
dataset = CovidPneumoniaDataset(root_dir=data_dir, transform=transform)

# Cross-validation parameters
n_splits = 5
num_epochs = 5
batch_size = 16
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Training and validation loop
fold_train_accs = []
fold_val_accs = []
fold_val_f1s = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
    print(f'\nFold {fold + 1}/{n_splits}')
    
    # Create data loaders
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    val_subset = torch.utils.data.Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    
    # Initialize model, optimizer, and loss
    model = SwinBinaryClassifier().to(device)
    optimizer = optim.Adam(model.parameters(), lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_preds, train_labels = [], []
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds) * 100
        avg_loss = running_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds) * 100
        val_f1 = f1_score(val_labels, val_preds, average='binary')
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Acc: {val_acc:.2f}%, Val F1: {val_f1:.4f}')
        
        scheduler.step()
    
    fold_train_accs.append(train_acc)
    fold_val_accs.append(val_acc)
    fold_val_f1s.append(val_f1)

# Print final results
print('\nCross-validation Results:')
print(f'Average Train Accuracy: {np.mean(fold_train_accs):.2f}% (±{np.std(fold_train_accs):.2f})')
print(f'Average Val Accuracy: {np.mean(fold_val_accs):.2f}% (±{np.std(fold_val_accs):.2f})')
print(f'Average Val F1 Score: {np.mean(fold_val_f1s):.4f} (±{np.std(fold_val_f1s):.4f})')

print("Training complete!")


Fold 1/5


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

Epoch 1/5:
Train Loss: 0.5683, Train Acc: 71.61%
Val Acc: 95.45%, Val F1: 0.9508
Epoch 2/5:
Train Loss: 0.1453, Train Acc: 96.20%
Val Acc: 98.99%, Val F1: 0.9888
Epoch 3/5:
Train Loss: 0.0492, Train Acc: 98.99%
Val Acc: 99.49%, Val F1: 0.9944
Epoch 4/5:
Train Loss: 0.0418, Train Acc: 99.49%
Val Acc: 99.49%, Val F1: 0.9944
Epoch 5/5:
Train Loss: 0.0417, Train Acc: 99.62%
Val Acc: 99.49%, Val F1: 0.9944

Fold 2/5
Epoch 1/5:
Train Loss: 0.5323, Train Acc: 76.55%
Val Acc: 94.95%, Val F1: 0.9444
Epoch 2/5:
Train Loss: 0.1116, Train Acc: 95.44%
Val Acc: 100.00%, Val F1: 1.0000
Epoch 3/5:
Train Loss: 0.0620, Train Acc: 98.61%
Val Acc: 99.49%, Val F1: 0.9942
Epoch 4/5:
Train Loss: 0.0456, Train Acc: 99.37%
Val Acc: 99.49%, Val F1: 0.9942
Epoch 5/5:
Train Loss: 0.0489, Train Acc: 98.99%
Val Acc: 99.49%, Val F1: 0.9942

Fold 3/5
Epoch 1/5:
Train Loss: 0.3873, Train Acc: 84.05%
Val Acc: 94.92%, Val F1: 0.9474
Epoch 2/5:
Train Loss: 0.1317, Train Acc: 94.81%
Val Acc: 93.40%, Val F1: 0.9326
Epoch 3

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from transformers import SwinModel, SwinConfig

# Custom Dataset Class
class CovidPneumoniaDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['covid', 'pneumonia']
        self.images = []
        self.labels = []
        
        for label, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.images.append(img_path)
                    self.labels.append(label)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Custom Model with Swin Tiny Transformer
class SwinBinaryClassifier(nn.Module):
    def __init__(self):
        super(SwinBinaryClassifier, self).__init__()
        self.swin = SwinModel.from_pretrained('microsoft/swin-tiny-patch4-window7-224')
        self.head = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )
    
    def forward_features(self, x):
        return self.swin(x).last_hidden_state[:, 0, :]
    
    def forward(self, x):
        features = self.forward_features(x)
        return self.head(features)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

# Load full dataset
data_dir = "/kaggle/input/covid-pneumonia-lus-images/covid_pneumonia"  # Replace with your dataset path containing covid/ and pneumonia/ folders
full_dataset = CovidPneumoniaDataset(root_dir=data_dir, transform=transform)

# Perform train-test split
train_idx, test_idx = train_test_split(
    range(len(full_dataset)),
    test_size=0.2,  # 20% for test set
    stratify=full_dataset.labels,  # Maintain class distribution
    random_state=42
)

train_dataset = Subset(full_dataset, train_idx)
test_dataset = Subset(full_dataset, test_idx)

# Cross-validation parameters
n_splits = 5
num_epochs = 5
batch_size = 16
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Training and validation loop for hyperparameter tuning
fold_val_accs = []
fold_val_f1s = []

for fold, (train_idx_cv, val_idx_cv) in enumerate(kfold.split(range(len(train_dataset)))):
    print(f'\nFold {fold + 1}/{n_splits}')
    
    # Create train and validation subsets from training data
    train_subset = Subset(train_dataset, train_idx_cv)
    val_subset = Subset(train_dataset, val_idx_cv)
    
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    
    model = SwinBinaryClassifier().to(device)
    optimizer = optim.Adam(model.parameters(), lr=2e-5)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_preds, train_labels = [], []
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        train_acc = accuracy_score(train_labels, train_preds) * 100
        avg_loss = running_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds) * 100
        val_f1 = f1_score(val_labels, val_preds, average='binary')
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Acc: {val_acc:.2f}%, Val F1: {val_f1:.4f}')
        
        scheduler.step()
    
    fold_val_accs.append(val_acc)
    fold_val_f1s.append(val_f1)

# Cross-validation results
print('\nCross-validation Results:')
print(f'Average Val Accuracy: {np.mean(fold_val_accs):.2f}% (±{np.std(fold_val_accs):.2f})')
print(f'Average Val F1 Score: {np.mean(fold_val_f1s):.4f} (±{np.std(fold_val_f1s):.4f})')

# Train final model on full training set
print('\nTraining final model on full training set...')
final_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
final_model = SwinBinaryClassifier().to(device)
optimizer = optim.Adam(final_model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

for epoch in range(num_epochs):
    final_model.train()
    running_loss = 0.0
    train_preds, train_labels = [], []
    
    for images, labels in final_train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_preds.extend(predicted.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())
    
    train_acc = accuracy_score(train_labels, train_preds) * 100
    avg_loss = running_loss / len(final_train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {avg_loss:.4f}, Train Acc: {train_acc:.2f}%')
    scheduler.step()

# Evaluate on test set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
final_model.eval()
test_preds, test_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds) * 100
test_f1 = f1_score(test_labels, test_preds, average='binary')
cm = confusion_matrix(test_labels, test_preds)

print('\nFinal Test Set Results:')
print(f'Test Accuracy: {test_acc:.2f}%')
print(f'Test F1 Score: {test_f1:.4f}')
print('Confusion Matrix:')
print(cm)
print("Training and evaluation complete!")


Fold 1/5
Epoch 1/5:
Train Loss: 0.4667, Train Acc: 78.92%
Val Acc: 87.34%, Val F1: 0.8750
Epoch 2/5:
Train Loss: 0.1601, Train Acc: 93.34%
Val Acc: 93.67%, Val F1: 0.9390
Epoch 3/5:
Train Loss: 0.1230, Train Acc: 94.93%
Val Acc: 94.94%, Val F1: 0.9506
Epoch 4/5:
Train Loss: 0.0766, Train Acc: 96.83%
Val Acc: 95.57%, Val F1: 0.9565
Epoch 5/5:
Train Loss: 0.0736, Train Acc: 97.31%
Val Acc: 95.57%, Val F1: 0.9565

Fold 2/5
Epoch 1/5:
Train Loss: 0.5038, Train Acc: 79.56%
Val Acc: 88.61%, Val F1: 0.8929
Epoch 2/5:
Train Loss: 0.2047, Train Acc: 90.33%
Val Acc: 97.47%, Val F1: 0.9747
Epoch 3/5:
Train Loss: 0.1051, Train Acc: 96.99%
Val Acc: 98.73%, Val F1: 0.9873
Epoch 4/5:
Train Loss: 0.1005, Train Acc: 97.46%
Val Acc: 100.00%, Val F1: 1.0000
Epoch 5/5:
Train Loss: 0.0808, Train Acc: 98.10%
Val Acc: 100.00%, Val F1: 1.0000

Fold 3/5
Epoch 1/5:
Train Loss: 0.4161, Train Acc: 83.52%
Val Acc: 95.57%, Val F1: 0.9560
Epoch 2/5:
Train Loss: 0.1705, Train Acc: 93.50%
Val Acc: 98.10%, Val F1: 0.9